In [1]:
# Install required libraries
!pip install -q streamlit pyngrok opencv-python

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Get ngrok authentication token
from pyngrok import ngrok
# Sign up for a free ngrok account to get your authtoken
# https://dashboard.ngrok.com/get-started/your-authtoken
# Replace "YOUR_NGROK_AUTHTOKEN_HERE" with your actual token
ngrok.set_auth_token("32o5vfP7i9N4C3agY5OjLa7flaE_7Ka8twedAty5WPjmv5KtD")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import cv2, os
import io
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Flatten, Dense
from tensorflow.keras import backend as K

# --- Configuration ---
path = "/content/drive/MyDrive/dataset"
image_folder = f"{path}/images/"
sequence_len = 6
image_size = (64, 64)
MAX_IMAGE_PIXELS = 178956970  # PIL default max pixel limit (approx 178 million)

# --- Define R2 metric (needed for loading model) ---
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

# --- Cached functions ---
@st.cache_resource
def load_trained_model():
    model = Sequential([
        ConvLSTM2D(filters=32, kernel_size=(3,3), activation='relu',
                   input_shape=(sequence_len, 64, 64, 3), return_sequences=True),
        BatchNormalization(),
        ConvLSTM2D(filters=32, kernel_size=(3,3), activation='relu', return_sequences=False),
        BatchNormalization(),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    # Uncomment if weights are available
    # model.load_weights("/content/drive/MyDrive/2019_09_07/model_weights.h5")
    return model

@st.cache_data
def load_irradiance_data():
    try:
        df = pd.read_csv(f"{path}/pyranometer.csv", header=None)
        df.columns = ["timestamp", "irradiance"]
        df["timestamp"] = pd.to_datetime(df["timestamp"], format="UTC-7_%Y_%m_%d-%H_%M_%S_%f", errors="coerce")
        df = df.set_index("timestamp").dropna()
        df = df[df["irradiance"] > 0]
        df = df.resample("5min").mean().dropna()
        return df
    except FileNotFoundError:
        st.error(f"Irradiance CSV not found at {path}/pyranometer.csv.")
        return None

# --- Safe image processing ---
def load_image_from_bytes(img_bytes, size=(64, 64)):
    try:
        img = Image.open(io.BytesIO(img_bytes))
        if img.width * img.height > MAX_IMAGE_PIXELS:
            st.error("🚫 Image too large. Please upload smaller images (e.g., <5000x5000).")
            return None
        img = img.resize(size)
        img_np = np.array(img)
        if img_np.ndim == 2:  # Grayscale
            img_np = np.stack([img_np]*3, axis=-1)
        elif img_np.shape[2] == 4:  # RGBA
            img_np = img_np[:, :, :3]
        return img_np / 255.0
    except Exception as e:
        st.error(f"Error loading image: {e}")
        return None

# --- Streamlit UI ---
st.title("☀️ ConvLSTM Irradiance Prediction")
st.markdown("""
Upload a sequence of 6 sky images to get a 30-minute irradiance forecast.

📌 **Note:** Images must be small (e.g., 64×64 to 512×512 pixels). Very large images will be rejected for safety.
""")

# Load resources
df_irradiance = load_irradiance_data()
model = load_trained_model()

# Upload input
uploaded_files = st.file_uploader("Choose 6 PNG or JPG images", type=["png", "jpg", "jpeg"], accept_multiple_files=True)

if uploaded_files:
    if len(uploaded_files) != 6:
        st.warning("⚠️ Please upload exactly 6 images.")
    else:
        st.subheader("Uploaded Sequence")
        cols = st.columns(6)
        image_seq = []

        for i, file in enumerate(uploaded_files):
            with cols[i]:
                st.image(file, caption=f"Image {i+1}", width=100)

            image_bytes = file.read()
            img = load_image_from_bytes(image_bytes, size=image_size)
            if img is not None:
                image_seq.append(img)
            else:
                st.error(f"Skipping image {file.name} due to size or format issues.")
                image_seq = None
                break

        if image_seq:
            image_seq_np = np.array(image_seq)
            if st.button("📈 Predict Irradiance"):
                with st.spinner("Predicting..."):
                    input_data = image_seq_np[np.newaxis, ...]
                    y_pred = model.predict(input_data)

                    try:
                        filename = os.path.basename(uploaded_files[0].name)
                        timestamp_str = filename.split(".")[0]
                        timestamp = pd.to_datetime(timestamp_str, format="UTC-7_%Y_%m_%d-%H_%M_%S_%f", errors="coerce")

                        if pd.notna(timestamp) and df_irradiance is not None:
                            forecast_timestamp = timestamp + pd.Timedelta(minutes=30)
                            nearest_idx = df_irradiance.index.get_indexer([forecast_timestamp], method='nearest')

                            if nearest_idx.size > 0 and nearest_idx.item() != -1:
                                true_value = df_irradiance.iloc[nearest_idx.item()]["irradiance"]

                                st.success(f"🤖 Predicted Irradiance (30 min ahead): {y_pred[0][0]:.2f} W/m²")
                                st.success(f"🌞 Actual Irradiance (at prediction time): {true_value:.2f} W/m²")

                                # --- Plot ---
                                fig, ax = plt.subplots()
                                labels = ['Predicted', 'Actual']
                                values = [y_pred[0][0], true_value]

                                ax.bar(labels, values, color=['skyblue', 'orange'])
                                ax.set_ylabel('Irradiance (W/m²)')
                                ax.set_title('Predicted vs Actual Irradiance')

                                for i, v in enumerate(values):
                                    ax.text(i, v + 5, f"{v:.2f}", ha='center', va='bottom')

                                st.pyplot(fig)
                            else:
                                st.info("Could not find corresponding actual irradiance value.")
                        else:
                            st.info("Could not parse timestamp or match irradiance.")
                    except Exception as e:
                        st.error(f"Error retrieving actual irradiance: {e}")

# Sidebar Info
st.sidebar.title("ℹ️ App Info")
st.sidebar.markdown("""
This app demonstrates a ConvLSTM model for solar irradiance prediction.

**Model:** ConvLSTM2D
**Input:** Sequence of 6 sky images
**Output:** Irradiance prediction 30 minutes into the future
""")


Overwriting app.py


In [3]:
import os
import threading
import subprocess
import time
from pyngrok import ngrok

# Get ngrok authentication token
# You must visit https://dashboard.ngrok.com/get-started/your-authtoken
# and replace the placeholder below with your actual token.
ngrok.set_auth_token("32o5vfP7i9N4C3agY5OjLa7flaE_7Ka8twedAty5WPjmv5KtD")

def run_streamlit():
    subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"])

print("Starting Streamlit app...")
threading.Thread(target=run_streamlit).start()
time.sleep(5)

public_url = ngrok.connect(8501)
print(f"Your Streamlit app is live at: {public_url}")



Starting Streamlit app...
Your Streamlit app is live at: NgrokTunnel: "https://e6610a1021ef.ngrok-free.app" -> "http://localhost:8501"
